## LLM as a JUDGE

In [14]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

### 평가받을 답변 3가지 정의

In [9]:
from langchain_community.chat_models import ChatOllama

llm_EEVE = ChatOllama(model="EEVE-Korean-Instruct-10.8B-v1.0-Q8_0:latest")
llm_Llama3 = ChatOllama(model="Llama-3-Open-Ko-8B-Q8_0:latest")
llm_EXAONE = ChatOllama(model="exaone-3.0-7.8b-it-Q8_0:latest")

In [10]:
query = "65세 여성 노인이 보건의료 관련해서 제주도에서 지원받을 수 있는 정책은 뭐가 있어? 지원받을 수 있는 정책을 다 알려줘"

In [11]:
EEVE_response = llm_EEVE.invoke(query)
EEVE_answer = EEVE_response.content

Llama3_response = llm_Llama3.invoke(query)
Llama3_answer = Llama3_response.content

EXAONE_response = llm_EXAONE.invoke(query)
EXAONE_answer = EXAONE_response.content

### 평가자 정의

In [15]:
from langchain_openai import ChatOpenAI

eval_llm = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    model_name="gpt-4o",  # 모델명
)

In [25]:
# LogicKor 의 평가 요구 템플릿

JUDGE_TEMPLATE = {
    "single_turn": """너는 질문에 대한 한국어 언어 모델의 답변을 매우 꼼꼼히 평가할 것이다. 공정한 평가를 위해 아래의 규칙을 준수한다.

# 기본 규칙
1. 질문의 요구사항을 충분히 반영하였는지 상세히 분석할 것.
2. 답변 과정에서 누락되었거나 포함되지 못하여 아쉬운 부분에 대하여 상세히 분석할 것.
3. 답변의 길이가 평가 결과에 영향을 미치지 않도록 할 것.
4. Additional Reference가 제공된다면 평가 시 해당 정보를 참고할 것.

# 언어 요구사항
- 모델은 반드시 한국어로 답변해야 하며, 다른 언어로의 답변은 절대 허용되지 않는다.
- 예외적으로 질문이 영어로 답변할 것을 요구할 때에만 영어 답변이 허용된다.
- 한국어로 답변하지 않을 경우, 점수는 0점 처리된다.
- 언어 요구사항을 충족하는 것은 필수적이나, 이 요구사항의 충족이 답변의 질적 평가에 추가 점수로 이어지지는 않는다.

# 평가 출력 방식
**주어진 Question에 집중하여** Model's Response에 대한 평가와 1~10의 점수를 부여한다. 답변에 대한 평가는 4~5 문장으로 규칙을 참고하여 상세히 작성한다.

[Question]
{question}

[The Start of Model's Response]
{model_answer}
[The End of Model's Response]

# 출력 형식
평가: 평가 내용
점수: 숫자""",
}

In [26]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template=JUDGE_TEMPLATE['single_turn'],
    input_variables=["question", "model_answer"]
)

In [27]:
# Function to evaluate each response
def evaluate_response(eval_llm, prompt, query, response):
    formatted_prompt = prompt.format_prompt(
        question=query,
        model_answer=response
    ).to_string()
    
    evaluation = eval_llm.invoke(formatted_prompt)
    return evaluation.content

In [28]:
# Evaluate each model's response
EEVE_evaluation = evaluate_response(eval_llm, prompt, query, EEVE_answer)
Llama3_evaluation = evaluate_response(eval_llm, prompt, query, Llama3_answer)
EXAONE_evaluation = evaluate_response(eval_llm, prompt, query, EXAONE_answer)

# Print evaluations
print("EEVE Evaluation:", EEVE_evaluation)
print("Llama3 Evaluation:", Llama3_evaluation)
print("EXAONE Evaluation:", EXAONE_evaluation)

EEVE Evaluation: 평가: 모델의 답변은 질문에서 요구한 65세 여성 노인이 제주도에서 받을 수 있는 보건의료 관련 지원 정책에 대해 매우 포괄적이고 상세하게 설명하고 있습니다. 다양한 정책과 프로그램을 구체적으로 나열하며, 각 항목에 대한 간단한 설명을 제공하여 이해를 돕고 있습니다. 다만, 각 정책의 구체적인 신청 방법이나 자격 요건에 대한 정보가 추가되었다면 더욱 완벽한 답변이 되었을 것입니다. 전반적으로 질문의 요구사항을 잘 반영한 답변입니다.

점수: 9
Llama3 Evaluation: 평가: 모델의 답변은 질문에 대한 요구사항을 어느 정도 충족하고 있습니다. 제주특별자치도에서 65세 이상 노인을 대상으로 제공하는 몇 가지 보건의료 지원 정책을 언급하였으며, 건강보험 본인부담금 지원과 노인장기요양보험법에 따른 요양서비스, 치매안심센터의 역할 등을 설명하였습니다. 그러나 질문에서 요구한 "모든" 정책을 다루기에는 정보가 부족하며, 다른 가능한 지원 정책이나 서비스에 대한 언급이 누락되었습니다. 예를 들어, 지역사회 건강 프로그램이나 기타 노인 복지 서비스에 대한 정보가 추가되었으면 더 완벽한 답변이 되었을 것입니다.

점수: 7
EXAONE Evaluation: 평가: 모델의 답변은 질문의 요구사항을 충분히 반영하여 제주도에서 65세 이상 노인이 지원받을 수 있는 다양한 보건의료 및 사회복지 정책을 상세히 설명하고 있습니다. 각 정책에 대한 설명이 구체적이며, 노인의 생활 안정, 건강 관리, 주거 환경 개선, 문화 및 여가 활동, 교통 지원 등 다양한 측면을 포괄하고 있습니다. 또한, 추가적인 정보나 상담을 받을 수 있는 방법도 안내하여 실질적인 도움이 될 수 있도록 하였습니다. 전반적으로 질문에 대한 답변이 충실하며, 누락된 부분 없이 잘 구성되어 있습니다.

점수: 10
